# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.6265704633495803
 2.084121705772402
 2.1667981597631303

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.360897   1.0       0.0
 0.934721  -0.608492  1.0
U factor:
3×3 Array{Float64,2}:
 0.994809  0.540932  0.548381
 0.0       0.649609  0.765037
 0.0       0.0       0.539299

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.360897   1.0       0.0
 0.934721  -0.608492  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.994809  0.540932  0.548381
 0.0       0.649609  0.765037
 0.0       0.0       0.539299

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 0.9999999999999998
 0.9999999999999997
 1.0000000000000002

In [8]:
Alu\b

3-element Array{Float64,1}:
 0.9999999999999998
 0.9999999999999997
 1.0000000000000002

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [12]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.660296   0.440469    0.608274
 -0.70641   -0.0893212  -0.702145
 -0.254941  -0.893314    0.37013
R factor:
3×3 Array{Float64,2}:
 -1.40826  -0.670357  -1.02005
  0.0      -0.754413  -0.65092
  0.0       0.0        0.328041

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [13]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.660296   0.440469    0.608274
 -0.70641   -0.0893212  -0.702145
 -0.254941  -0.893314    0.37013

In [14]:
Aqr.R

3×3 Array{Float64,2}:
 -1.40826  -0.670357  -1.02005
  0.0      -0.754413  -0.65092
  0.0       0.0        0.328041

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [15]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.037537991919875946
  0.9660218222523929
  3.9390079966397553
vectors:
3×3 Array{Float64,2}:
  0.253687  -0.783139  -0.567747
 -0.842319   0.109703  -0.527697
  0.475543   0.612094  -0.631823

The values and the vectors can be extracted from the Eigen type by special indexing

In [16]:
AsymEig.values

3-element Array{Float64,1}:
 -0.037537991919875946
  0.9660218222523929
  3.9390079966397553

In [17]:
AsymEig.vectors

3×3 Array{Float64,2}:
  0.253687  -0.783139  -0.567747
 -0.842319   0.109703  -0.527697
  0.475543   0.612094  -0.631823

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [18]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0          -1.33227e-14   2.4869e-14
  1.77636e-14   1.0          -7.81597e-14
 -1.24345e-14  -2.39808e-14   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [19]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [20]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [21]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

3.367307920699754

In [22]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [23]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [24]:
@time eigvals(Asym);

  0.304263 seconds (168.06 k allocations: 16.063 MiB, 2.55% gc time)


In [25]:
@time eigvals(Asym_noisy);

  0.790252 seconds (14 allocations: 7.921 MiB)


In [26]:
@time eigvals(Asym_explicit);

  0.195948 seconds (7.72 k allocations: 8.387 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `10x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [27]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.744918 seconds (473.51 k allocations: 206.850 MiB, 13.93% gc time)


6.241422099286945

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [28]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [29]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 9//10  4//5   3//10
 2//5   1//10  1//1
 2//5   9//10  1//1

In [30]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
  2//1
  3//2
 23//10

In [31]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [32]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1    0//1   0//1
 4//9    1//1   0//1
 4//9  -23//49  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 9//10   4//5     3//10
 0//1   49//90   13//15
 0//1    0//1   312//245

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [34]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [35]:
A_eigv=eigen(A).values

5-element Array{Float64,1}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137

In [36]:
@assert A_eigv ≈  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

In [37]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

AssertionError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [42]:
using LinearAlgebra, Statistics

In [46]:
?diagm

search: diagm diag diagind Diagonal isdiag Bidiagonal Tridiagonal SymTridiagonal



```
diagm(kv::Pair{<:Integer,<:AbstractVector}...)
diagm(m::Integer, n::Integer, kv::Pair{<:Integer,<:AbstractVector}...)
```

Construct a matrix from `Pair`s of diagonals and vectors. Vector `kv.second` will be placed on the `kv.first` diagonal. By default the matrix is square and its size is inferred from `kv`, but a non-square size `m`×`n` (padded with zeros as needed) can be specified by passing `m,n` as the first arguments.

`diagm` constructs a full matrix; if you want storage-efficient versions with fast arithmetic, see [`Diagonal`](@ref), [`Bidiagonal`](@ref) [`Tridiagonal`](@ref) and [`SymTridiagonal`](@ref).

# Examples

```jldoctest
julia> diagm(1 => [1,2,3])
4×4 Array{Int64,2}:
 0  1  0  0
 0  0  2  0
 0  0  0  3
 0  0  0  0

julia> diagm(1 => [1,2,3], -1 => [4,5])
4×4 Array{Int64,2}:
 0  1  0  0
 4  0  2  0
 0  5  0  3
 0  0  0  0
```

---

```
diagm(v::AbstractVector)
diagm(m::Integer, n::Integer, v::AbstractVector)
```

Construct a matrix with elements of the vector as diagonal elements. By default (if `size=nothing`), the matrix is square and its size is given by `length(v)`, but a non-square size `m`×`n` can be specified by passing `m,n` as the first arguments.

# Examples

```jldoctest
julia> diagm([1,2,3])
3×3 Array{Int64,2}:
 1  0  0
 0  2  0
 0  0  3
```


In [47]:
D=diagm([1,2,3])

3×3 Array{Int64,2}:
 1  0  0
 0  2  0
 0  0  3

In [48]:
A_diag=diagm(A_eigv)

5×5 Array{Float64,2}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [49]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

AssertionError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [50]:
?LowerTriangular

search: LowerTriangular UnitLowerTriangular



```
LowerTriangular(A::AbstractMatrix)
```

Construct a `LowerTriangular` view of the matrix `A`.

# Examples

```jldoctest
julia> A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 9.0]
3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

julia> LowerTriangular(A)
3×3 LowerTriangular{Float64,Array{Float64,2}}:
 1.0   ⋅    ⋅
 4.0  5.0   ⋅
 7.0  8.0  9.0
```


In [51]:
A_lowertri=LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [52]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

### Please let us know how we're doing!

Please execute the @assert cell once you are done with the exercises to validate your solution.